In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Ali Sobhani Thesis/TS'

/content/drive/.shortcut-targets-by-id/10ul3ZK_MC4nZuEgYotLTtUO0eEZB3gue/Ali Sobhani Thesis/TS


In [3]:
!pip install scikit-optimize
!pip install scikeras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.6 MB/s eta 0:00:00


In [4]:
from tensorflow.keras.layers import Flatten, Conv2D, DepthwiseConv2D, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

X = np.load("X.npy", allow_pickle=True)
Y = np.load("Y.npy")

def create_model1(optimizer='adam', kernel_size=3, dropout=0.5, norm_rate=0.01, F1=16, D=2, activation='relu'):
    input1       = Input(shape = (8, 7, 1))
    block1       = Conv2D(F1, (1, kernel_size), padding = 'same',
                                    input_shape = (1, 8, 7),
                                    use_bias = False)(input1)
    block1       = BatchNormalization(axis = -1)(block1)
    block2       = DepthwiseConv2D((8, 1), use_bias = False,
                                      depth_multiplier = D,
                                      depthwise_constraint = max_norm(norm_rate))(block1)
    block2       = BatchNormalization(axis = -1)(block2)
    if activation == 'leakyrelu':
        block2   = LeakyReLU()(block2)
    else:
        block2   = Activation(activation)(block2)
    flatten      = Flatten(name = 'flatten')(block2)
    block3       = Dropout(dropout)(flatten)
    dense        = Dense(1, name = 'dense',
                          kernel_constraint = max_norm(norm_rate))(block3)
    sigmoid      = Activation('sigmoid', name = 'sigmoid')(dense)
    model        = Model(inputs=input1, outputs=sigmoid)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

p = {
    'optimizer': ['Adam', 'Adamax', 'Nadam'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 15, 20, 30],
    'kernel_size': [2, 3, 4, 5],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'norm_rate': [0.01, 0.001, 0.1, 1, 5, 10],
    'F1': [16, 32, 64],
    'D': [1, 2, 3],
    'activation': ['relu', 'elu', 'leakyrelu']
}

model = KerasClassifier(model=create_model1, verbose=1, **p)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=13)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)

random = HalvingRandomSearchCV(estimator=model, param_distributions=p, n_candidates=50, resource='n_samples', cv=inner_cv, scoring='accuracy')

best_scores = {'accuracy': [], 'f2': [], 'f1':[]}
best_params = []

for train_idx, test_idx in outer_cv.split(X, Y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    random_result = random.fit(X_train, y_train)

    best_params.append(random_result.best_params_)
    #best_scores['accuracy'].append(bayes_result.best_score_)
    y_pred = random.predict(X_test)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    ################################
    from sklearn.metrics import f1_score, accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    ################################
    best_scores['f2'].append(f2)
    best_scores['f1'].append(f1)
    best_scores['accuracy'].append(accuracy)
average_best_scores = {scoring: np.mean(scores) for scoring, scores in best_scores.items()}
print(f"Average best scores: {average_best_scores}")


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 0.6960 - accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 0.7396 - accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 0.7511 - accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 0s 11ms/step - loss: 0.5938 - accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 0.3904 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.5662 - accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 0.5936 - accuracy: 0.6667
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.3391 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.3354 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 18ms/step
Epoch 1/10
1/1 [==============================] - 2s 2s/step - 

1/1 [==============================] - 0s 20ms/step
Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 0.5818 - accuracy: 0.7500
Epoch 2/15
1/1 [==============================] - 0s 13ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 4/15
1/1 [==============================] - 0s 12ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 5/15
1/1 [==============================] - 0s 13ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 6/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 7/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 8/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 9/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 10/15
1/1 [==============================] - 0s 11ms/step - loss: 0.693

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 19ms/step
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.4896 - accuracy: 0.6667
Epoch 2/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6934 - accuracy: 0.3333
Epoch 3/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6916 - accuracy: 0.6667
Epoch 4/20
1/1 [==============================] - 0s 15ms/step - loss: 0.6905 - accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6905 - accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6896 - accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6890 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6886 - accuracy: 0.6667
Epoch 9/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6886 - accuracy: 0.6667
Epoch 10/20
1/1 [=========

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step
Epoch 1/15
1/1 [==============================] - 3s 3s/step - loss: 0.8194 - accuracy: 0.5000
Epoch 2/15
1/1 [==============================] - 0s 18ms/step - loss: 0.4314 - accuracy: 1.0000
Epoch 3/15
1/1 [==============================] - 0s 20ms/step - loss: 0.6439 - accuracy: 0.5000
Epoch 4/15
1/1 [==============================] - 0s 15ms/step - loss: 0.4613 - accuracy: 0.7500
Epoch 5/15
1/1 [==============================] - 0s 14ms/step - loss: 0.3039 - accuracy: 1.0000
Epoch 6/15
1/1 [==============================] - 0s 12ms/step - loss: 0.7736 - accuracy: 0.2500
Epoch 7/15
1/1 [==============================] - 0s 15ms/step - loss: 0.4809 - accuracy: 1.0000
Epoch 8/15
1/1 [==============================] - 0s 13ms/step - loss: 0.4276 - accuracy: 1.0000
Epoch 9/15
1/1 [==============================] - 0s 14ms/step - loss: 0.2167 - accuracy: 1.0000
Epoch 10/15
1/1 [=========

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Streaming output truncated to the last 5000 lines.
Epoch 20/20
1/1 [==============================] - 0s 31ms/step
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.5355 - accuracy: 0.7500
Epoch 2/20
1/1 [==============================] - 0s 19ms/step - loss: 0.8141 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 15ms/step - loss: 0.3338 - accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 14ms/step - loss: 0.3482 - accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 17ms/step - loss: 0.1900 - accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 17ms/step - loss: 0.5798 - accuracy: 0.5000
Epoch 7/20
1/1 [==============================] - 0s 13ms/step - loss: 0.2832 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 15ms/step - loss: 0.2546 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 12ms/step - loss: 0.2442 - accuracy: 1.0000
Epoch 10/20
1/

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 19ms/step
Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 1.1540 - accuracy: 0.2500
Epoch 2/15
1/1 [==============================] - 0s 12ms/step - loss: 0.6926 - accuracy: 0.7500
Epoch 3/15
1/1 [==============================] - 0s 15ms/step - loss: 0.6755 - accuracy: 1.0000
Epoch 4/15
1/1 [==============================] - 0s 10ms/step - loss: 0.6721 - accuracy: 1.0000
Epoch 5/15
1/1 [==============================] - 0s 10ms/step - loss: 0.6665 - accuracy: 1.0000
Epoch 6/15
1/1 [==============================] - 0s 13ms/step - loss: 0.6619 - accuracy: 1.0000
Epoch 7/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6585 - accuracy: 1.0000
Epoch 8/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6548 - accuracy: 1.0000
Epoch 9/15
1/1 [==============================] - 0s 11ms/step - loss: 0.6515 - accuracy: 1.0000
Epoch 10/15
1/1 [=========

In [5]:
import pandas as pd
import openpyxl
model = ['Model1', 'Model2', 'Model3', 'Model4']
FS = ['Bayes', 'Random', 'Grid']
# Load the existing file
book = openpyxl.load_workbook('TS_Results.xlsx')

# Prepare the data to be written
data_acc = average_best_scores['accuracy']
data_f2 = average_best_scores['f2']
data_f1 = average_best_scores['f1']

# Get the existing sheets
sheet_acc = book['ACC']
sheet_f2 = book['F2']
sheet_f1 = book['F1']

# Calculate the correct row and column numbers
row = model.index('Model1') + 2  # +2 because Excel index starts from 1 and row 1 contains headers
col = FS.index('Random') + 2  # +2 because Excel index starts from 1 and column 1 contains headers

# Write to the ACC sheet
sheet_acc.cell(row=row, column=col, value=data_acc)

# Write to the F2 sheet
sheet_f2.cell(row=row, column=col, value=data_f2)

# Write to the F1 sheet
sheet_f1.cell(row=row, column=col, value=data_f1)

# Save and close the Excel file
book.save('TS_Results.xlsx')

In [6]:
import os
import pickle

# Specify the directory path
dir_path = '/content/drive/MyDrive/Ali Sobhani Thesis/TS/Random/Model1'

# Save best_params and all_selected_names to the directory
with open(os.path.join(dir_path, 'best_params.pkl'), 'wb') as f:
    pickle.dump(best_params, f)